In [53]:
from collections import defaultdict, namedtuple
from abc import ABC, abstractmethod
import random
from collections import defaultdict
import random
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt

In [54]:
class Node:
    def __init__(self, id) -> None:
        self.id = id
        self.connections = defaultdict(list)

    def get_edge_costs(self, other_node: 'Node'):
        return self.connections[other_node]
    
    def have_connection(self, other_node: 'Node'):
        return len(self.connections[other_node]) != 0
    
    def __str__(self) -> str:
        return f"Node: id={self.id}"

In [55]:
class NodesGroup:
    def __init__(self) -> None:
        self.nodes = {} # Id вершины: объект вершины

    def add_node(self, node: Node, replace=False):
        if node.id in self.nodes and replace is False:
            return False
        self.nodes[node.id] = node

    def add_edge(self, first_node: 'Node', second_node: 'Node', *costs):
        if second_node.id not in self.nodes:
            raise ValueError(f"В NodesGroup нет одной из вершин: {first_node} {second_node}")
        self.nodes[first_node.id].connections[second_node.id] = costs

    def add_nodes(self, *nodes, replace=False):
         for node in nodes:
              if type(node) != Node:
                   raise TypeError(f"Объект{node}: типа {type(node)} нельзя добавить в NodeGroup")
              if node.id in self.nodes and replace is False:
                   continue
              self.nodes[node.id] = node

    def delete_node(self, node: Node):
            self.nodes.pop(node.id)

    def __getitem__(self, key: int):
        if type(key) != int:
            raise TypeError("Обращаться к NodeGroup можно только по id ноды (тип int)")
        if key not in self.nodes:
            raise KeyError(f"В NodeGroup нет ноды с id:{key}")
        return self.nodes[key]
    
    def __iter__(self):
        return iter(self.nodes.values())
    
    def __len__(self):
         return len(self.nodes)
    
    def extend(self, other_nodes_group: 'NodesGroup'):
         for node_id in other_nodes_group.nodes:
              if node_id not in self.nodes:
                   self.nodes[node_id] = other_nodes_group[node_id]

    def get_nodes(self):
        return list(self.nodes.values())


In [57]:
class TestBuilder(ABC):
    @abstractmethod
    def build_map(self):
        pass

    @abstractmethod
    def build_test(self):
        pass

In [58]:
class TestBuildDirector:
    def __init__(self, builder: TestBuilder) -> None:
        self.builder = builder
    
    def construct_test(self):
        self.builder.build_map()
        self.builder.build_solution()
        self.builder.build_test()

    def change_builder(self, new_builder: TestBuilder) -> None:
        self.builder = new_builder

In [59]:
class SimpleTestBuilder(TestBuilder):
    def __init__(self, output_file_name) -> None:
        self.nodes_group = NodesGroup()
        self.output_file_name = output_file_name
        self.test_full_text = ""
        self.solutuion_info = ""
    
    def build_map(self):
        pass

    def build_solution(self):
        pass

    def build_test(self):
        edges_formated = []
        for node in self.nodes_group:
            for edge_and_cost in node.connections.items():
                edges_formated.append(f"{node.id} {edge_and_cost[0]} {' '.join(map(str, edge_and_cost[1]))}")
        test_info = f"{len(self.nodes_group)} {len(edges_formated)}"

        test_nodes_and_edges_info = "\n".join([test_info, *edges_formated])

        self.test_full_text = "\n".join([self.solutuion_info, test_nodes_and_edges_info])
        return self.test_full_text

    def get_test(self):
        if not self.test_full_text:
            return ValueError("Тест ещё не был создан") 
        return self.test_full_text

In [60]:
class Graph:
    def __init__(self):
        self.adjacency_list = defaultdict(dict)
        self.vertices = set()

    def add_edge(self, vertex1, vertex2, cost1, cost2):
        self.adjacency_list[vertex1][vertex2] = (cost1, cost2)
        self.vertices.update([vertex1, vertex2])

    def read_from_file(self, file_path):
        self.adjacency_list = defaultdict(dict)
        self.vertices = set()
        try:
            with open(file_path, 'r') as file:
                for line in file:
                    vertex1, vertex2, cost1, cost2 = map(int, line.split())
                    self.add_edge(vertex1, vertex2, cost1, cost2)
        except FileNotFoundError:
            print(f"File {file_path} not found.")

    def print_graph(self):
        print("  V1 -> V2  :  C1, C2")
        for vertex, edges in self.adjacency_list.items():
            for neighbor, costs in edges.items():
                print(f"{vertex:>4} -> {neighbor:<4}: {costs[0]:>3}, {costs[1]}")

    def get_neighbors(self, state):
        if state in self.adjacency_list:
            return list(self.adjacency_list[state].items())
        return []
    

class Solution(ABC):
    @abstractmethod
    def __init__(self, solution_values):
        pass

    @abstractmethod
    def dominates(self, other):
        pass

class BiObjSolution(Solution):
    def __init__(self, solution_values, path):
        self.solution_values = tuple(solution_values)
        self.g1 = solution_values[0]
        self.g2 = solution_values[1]
        self.path = path

    def dominates(self, other: 'BiObjSolution'):
        return (self.g1 < other.g1 and self.g2 <= other.g2) or (self.g1 <= other.g1 and self.g2 < other.g2)
    
    def is_dominated_by(self, other: 'BiObjSolution'):
        return not self.dominates(other)

    def __str__(self):
        return f"({self.g1}, {self.g2})"

    def __hash__(self) -> int:
        return hash((self.g1, self.g2))

    def __repr__(self) -> str:
        return f"BiObjSolution([{self.g1}, {self.g2}])"

    def __eq__(self, other: 'BiObjSolution') -> bool:
        return isinstance(other, BiObjSolution) and self.g1 == other.g1 and self.g2 == other.g2


class ParetoSet:
    def __init__(self, SolutionClass=BiObjSolution):
        self.solutions = set()
        self.SolutionClass = SolutionClass

        #Для визуализации
        self.max_y = 0
        self.max_x = 0
        self.all_solusions_ever = set()


    def add_solution(self, solution_values, path):
        solution = self.SolutionClass(solution_values, path)
        non_dominated = self._is_non_dominated(solution)

        #Для визуализации
        self.all_solusions_ever.add(solution)
        self.max_x= max(self.max_x, solution.solution_values[0])
        self.max_y = max(self.max_y, solution.solution_values[1])

        if non_dominated:
            
            self.solutions = {s for s in self.solutions if not solution.dominates(s)}
            self.solutions.add(solution)
            return True
        
        return False

    def remove_solution(self, solution_values):
        solution = self.SolutionClass(solution_values)
        self.solutions.remove(solution)

    def _is_non_dominated(self, solution):
        return not any(s.dominates(solution) for s in self.solutions)

    def is_non_dominated(self, solution_values):
        solution = self.SolutionClass(solution_values)
        return not any(s.dominates(solution) for s in self.solutions)

    def get_solutions(self,values=True):
        if values:
            return {s.solution_values for s in self.solutions}
        return self.solutions

    def solutions_dominated_by(self, solution_values):
        solution = self.SolutionClass(solution_values)
        return [s for s in self.solutions if solution.dominates(s)]

    def _solutions_dominated_by(self, solution):
        return [s for s in self.solutions if solution.dominates(s)]

    def remove_worse(self, better_solution_values):
        better_solution = self.SolutionClass(better_solution_values)
        self.solutions.difference_update(self._solutions_dominated_by(better_solution))

    def visualize(self, color='blue', marker='o'):
        if not self.solutions:
            print("Empty Pareto set. Nothing to visualize.")
            return
        x_values, y_values = zip(*[(sol.g1, sol.g2) for sol in self.solutions])
        plt.scatter(x_values, y_values, label='Pareto Set', color=color, marker=marker, s=100, edgecolors='black')
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.title('Pareto Set Visualization')
        plt.grid(True, linestyle='--', which='both', alpha=0.7)
        plt.show()

    def __str__(self) -> str:
        return f"({', '.join(str(sol) for sol in self.solutions)})"

    def __contains__(self, solution_values):
        solution = self.SolutionClass(solution_values)
        return solution in self.solutions
    
def calculate_path_cost(graph, path):
    cost = [0, 0]
    for i in range(0, len(path)-1):
        c1, c2 = graph[path[i]][path[i+1]]
        cost[0] += c1
        cost[1] += c2
    return cost

In [61]:
def find_all_paths(graph, start, end):
    pareto_set_solution = ParetoSet() 
    stack = [(start, [start], {start})]
    paths = []
    true_paths = []

    while stack:
        current, path, visited = stack.pop()

        if current == end:
            paths.append(path)
            cost = [0, 0]
            for i in range(0, len(path)-1):
                c1, c2 = graph[path[i]][path[i+1]]
                cost[0] += c1
                cost[1] += c2
            if pareto_set_solution.add_solution(cost, path):
                true_paths.append(path)
            continue

        if graph[current]: # т.к graph - defaultdict
            for neighbor in graph[current]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    stack.append((neighbor, path + [neighbor], visited))

    return paths, true_paths, pareto_set_solution

In [62]:
class RandomTestsBuilder(SimpleTestBuilder):
    def __init__(self, output_file_name, nodes_count, edges_per_node_count) -> None:
        super().__init__(output_file_name)
        self.nodes_count = nodes_count
        self.edges_per_node_count = edges_per_node_count
        if edges_per_node_count > nodes_count:
            raise ValueError("Количество ребёр одной вершины не может превышать общее число вершин")
    
    def build_map(self):
        for node_id in range(self.nodes_count):
            self.nodes_group.add_node(Node(node_id))
        
        for cur_node in self.nodes_group:
            connections_count = random.randint(0, self.edges_per_node_count)
            choosed_nodes = random.sample(self.nodes_group.get_nodes(), connections_count)
            for neighbour_node in choosed_nodes:
                cost1 = round(random.random(), 5)
                cost2 = round(random.random(), 5)

                cost1 = random.randint(1, 100)
                cost2 = random.randint(1, 100)
                self.nodes_group.add_edge(cur_node, neighbour_node, cost1, cost2)

    def build_solution(self):
        graph = Graph()
        for node in self.nodes_group:
            for neighbour_node in node.connections:
                cost1, cost2 = node.connections[neighbour_node]
                graph.add_edge(node.id, neighbour_node, cost1, cost2)

        counter = 0
        while True:
            start, stop = random.sample(list(self.nodes_group.nodes.keys()), 2)
            paths, true_paths, pareto_set_solution = find_all_paths(graph.adjacency_list, start, stop)
            if len(true_paths) != 0 or counter >= 15:
                break
            counter += 1
            
        self.solutuion_info = f"{start} {stop} {len(pareto_set_solution.solutions)}"
        for sol in pareto_set_solution.solutions:
            self.solutuion_info += f"\n{' '.join(map(str, sol.solution_values))} {' '.join(map(str, sol.path))}"

    def build_test(self):
        return super().build_test()
    
    def reset(self):
        self.nodes_group = NodesGroup()
        self.test_full_text = ""
        self.solutuion_info = ""

In [65]:
for i in range(1):
    test_file = f"../tests/random_{i}.txt"
    with open(test_file, "r") as f:
        start, stop, sols_count = map(int, f.readline().split())
        sols = []
        for _ in range(sols_count):
            cost1, cost2, *path = map(int, f.readline().split())
            sols.append(((cost1, cost2), path))
        
        nodes_count, edges_count = map(int, f.readline().split())
        graph = Graph()
        for _ in range(edges_count):
            n1, n2, c1, c2 = map(int, f.readline().split())
            graph.add_edge(n1, n2, c1, c2)
        b_sols, good_paths, discarded_paths = BDijkstra(graph, start, search_tree=SearchTreePQD)
        print(b_sols[stop])
        print(sols)
